In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.table as table
from astropy.table import Table
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import os

# Dias cluster tables
* Create cluster member tables using FTP
* Cross match with NSS to create candidate pool
* Query cluster parameters from Vizier

In [34]:
## Constructing Dias cluster member tables using FTP/Vizier

# from ftplib import FTP

# clusters = Vizier(catalog='J/MNRAS/504/356',columns=['Cluster','N','logage','e_loagage','[Fe/H]','e_[Fe/H]','Av','e_Av','FileName'],row_limit=-1).query_constraints()[0]

# cut1 = [fname.startswith('clusters1') for fname in clusters['FileName']]
# cut2 = [fname.startswith('clusters2') for fname in clusters['FileName']]

# ftp = FTP('cdsarc.u-strasbg.fr')
# ftp.login()
# ftp.cwd('pub/cats/J/MNRAS/504/356/clusters2')
# for fname in tqdm(clusters[cut2]['FileName'],total=len(clusters[cut2])):
#     dirname,fname = fname.split('/') 
#     with open(os.path.join('data','clusters2',fname), 'wb') as f:
#         ftp.retrbinary('RETR ' + fname, f.write)
#     if dirname == 'clusters1':
#         new_names= [
#             "pmRApmDEcor",
#             "PlxpmRAcor",
#             "PlxpmDEcor",
#             "RFG",
#             "RFBP",
#             "RFRP",
#             "E(BR/RP)",
#             "Nper",
#             "[Fe/H]temp",
#             "RAdeg",
#             "e_RAdeg",
#             "DEdeg",
#             "e_DEdeg",
#             "Source",
#             "Plx",
#             "e_Plx",
#             "pmRA",
#             "e_pmRA",
#             "pmDE",
#             "e_pmDE",
#             "Dup",
#             "FG",
#             "e_FG",
#             "Gmag",
#             "e_Gmag",
#             "FBP",
#             "e_FBP",
#             "BPmag",
#             "e_BPmag",
#             "FRP",
#             "e_FRP",
#             "RPmag",
#             "e_RPmag",
#             "BP-RP",
#             "RV",
#             "e_RV",
#             "Teff",
#             "AG",
#             "E(BR-RP)",
#             "Rad",
#             "Lum",
#             "Pmemb",
#             "Cluster"
#         ]
#     elif dirname == 'clusters2':
#         new_names = [
#             "pmRApmDEcor",
#             "PlxpmRAcor",
#             "PlxpmDEcor",
#             "RFG",
#             "RFBP",
#             "RFRP",
#             "E(BR/RP)",
#             "Nper",
#             "RAdeg",
#             "e_RAdeg",
#             "DEdeg",
#             "e_DEdeg",
#             "Source",
#             "Plx",
#             "e_Plx",
#             "pmRA",
#             "e_pmRA",
#             "pmDE",
#             "e_pmDE",
#             "Dup",
#             "FG",
#             "e_FG",
#             "Gmag",
#             "e_Gmag",
#             "FBP",
#             "e_FBP",
#             "BPmag",
#             "e_BPmag",
#             "FRP",
#             "e_FRP",
#             "RPmag",
#             "e_RPmag",
#             "BP-RP",
#             "RV",
#             "e_RV",
#             "Teff",
#             "AG",
#             "E(BR-RP)",
#             "Rad",
#             "Lum",
#             "Pmemb"
#         ]
#     memb = pd.read_fwf(os.path.join('data','clusters2',fname),header=None)
#     if memb.shape[1] != len(new_names):
#         continue
#     memb.columns = new_names
#     memb = Table.from_pandas(memb)
#     memb.keep_columns(['Source','Pmemb'])
#     cname = fname.split('.')[0]
#     os.remove(os.path.join('data','clusters2',fname))

#     memb = memb[memb['Pmemb'] >= 0.9]
#     if len(memb) > 1:
#         query = '''SELECT source_id, ra , dec, ra_error ,dec_error,parallax,parallax_error, ruwe, phot_g_mean_flux, phot_g_mean_flux_error, phot_g_mean_mag,
#         phot_g_mean_flux_over_error, phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_flux_over_error, phot_bp_mean_mag, phot_rp_mean_flux,
#             phot_rp_mean_flux_error, phot_rp_mean_flux_over_error, phot_rp_mean_mag, bp_rp, bp_g, g_rp, radial_velocity, radial_velocity_error, l, b,
#             has_xp_sampled, has_rvs, has_epoch_photometry, has_epoch_rv, non_single_star, pmra, pmra_error, pmdec, pmdec_error
#             FROM gaiadr3.gaia_source
#             WHERE source_id IN {id_lst}
#             '''.format(id_lst = tuple(memb['Source'].data))
#         job = Gaia.launch_job(query= query)
#         memb1 = job.get_results()
#         if 'SOURCE_ID' in memb1.colnames:
#             memb1.rename_column('SOURCE_ID','source_id')
#         memb.rename_column('Source','source_id')
#         memb = table.join(memb,memb1,keys='source_id',join_type='inner')
#         cut = memb['parallax']/memb['parallax_error'] > 10
#         memb = memb[cut]
#         memb.write(os.path.join('data','clusters2',cname + '.fits'),format='fits',overwrite=True)
# ftp.quit()


100%|██████████| 478/478 [10:13<00:00,  1.28s/it]


'221 Goodbye.'

In [ ]:
## Creating a NSS table from Dias cluster members

# from os import walk

# basedir = './data/clusters2/'
# filenames = next(walk(basedir), (None, None, []))[2]
# # filenames.remove('cluster_ages.fits')
# # filenames.remove('cluster_ages.dat')

# clstr_lst = []
# id_lst = []
# for f in tqdm(filenames):
#     clstrname = f.split('.')[0]
#     tbl = Table.read(os.path.join(basedir,f), format='fits')
#     tbl = tbl[tbl['non_single_star'] > 0]
#     clstr_lst.extend(np.full(len(tbl), clstrname))
#     id_lst.extend(tbl['source_id'].data)
# table_0 = Table({'source_id': id_lst, 'cluster': clstr_lst})

In [ ]:
## Adding cluster ages, metallicities and extinctions to the NSS table
 
# clusters = Vizier(catalog='J/MNRAS/504/356',columns=['Cluster','N','logage','e_logage','[Fe/H]','e_[Fe/H]','Av','e_Av','FileName'],row_limit=-1).query_constraints()[0]

# clusters.rename_columns(clusters.colnames,['Cluster','N','logage','e_logage','Fe/H','e_Fe/H','Av','e_Av','FileName'])

# table_0['age'] = np.full(len(table_0),np.nan)
# table_0['age_err'] = np.full(len(table_0),np.nan)
# table_0['mh_for_mass_interp'] = np.full(len(table_0),np.nan)
# table_0['mh_err'] = np.full(len(table_0),np.nan)
# table_0['av_for_mass_interp'] = np.full(len(table_0),np.nan)
# table_0['av_err'] = np.full(len(table_0),np.nan)

# for i in range(len(table_0)):
#     cname = table_0[i]['cluster']
#     if cname not in clusters['Cluster']:
#         continue
#     age = 10**clusters[clusters['Cluster'] == cname]['logage']*1e-6
#     age_err = clusters[clusters['Cluster'] == cname]['e_logage'][0] * age / np.log(10)
#     feh = clusters[clusters['Cluster'] == cname]['Fe/H'][0]
#     feh_err = clusters[clusters['Cluster'] == cname]['e_Fe/H'][0]
#     av = clusters[clusters['Cluster'] == cname]['Av'][0]
#     av_err = clusters[clusters['Cluster'] == cname]['e_Av'][0]

#     table_0[i]['age'] = age
#     table_0[i]['age_err'] = age_err
#     table_0[i]['mh_for_mass_interp'] = feh
#     table_0[i]['mh_err'] = feh_err
#     table_0[i]['av_for_mass_interp'] = av
#     table_0[i]['av_err'] = av_err
    

# Hunt cluster table
* Cross match Hunt and NSS
* Apply cuts to the tables (membership prob, astrometric fidelity, cluster reliability)

In [2]:
## Loading Hunt clusters

# table_0 = Table.read('table_cut.fits', format='fits')
# table_0 = Table.read('table_B.fits', format='fits')
# table_1 = Table.read('table.fits', format='fits')
# members = Table.read('./data/hunt_clusters/members.csv', format='csv')
# clusters = Table.read('./data/hunt_clusters/clusters.csv', format='csv')

# cut = np.isin(table_0['source_id'], table_1['source_id']) | (table_0['cst'] < 5) | (table_0['class_50'] < 0.5) | (table_0['probability'] < 0.99)
# table_0 = table_0[~cut]

In [ ]:
# ## Get cluster data for each source
# members = Table.from_pandas(pd.read_csv('./data/hunt_clusters/members.csv',usecols=['source_id','id','probability']))
# clusters = Table.from_pandas(pd.read_csv('./data/hunt_clusters/clusters.csv',usecols=['id','log_age_16','log_age_50','log_age_84','a_v_16','a_v_50','a_v_84','n_stars']))

# table_0 = table.join(table_0, members, keys='source_id', join_type='left')
# table_0 = table.join(table_0, clusters, keys='id', join_type='left')

# Gaia and NSS parameters
* Add Gaia/NSS parameters to the candidate table

In [4]:
## query gaia for all sources in table_0- get relevant columns

query1 = '''SELECT source_id, ra , dec, ra_error ,dec_error,parallax,parallax_error, astrometric_excess_noise, astrometric_excess_noise_sig,
 astrometric_params_solved, pseudocolour, visibility_periods_used, astrometric_sigma5d_max, ruwe, phot_g_mean_flux, phot_g_mean_flux_error, phot_g_mean_mag,
   phot_g_mean_flux_over_error, phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_flux_over_error, phot_bp_mean_mag, phot_rp_mean_flux,
     phot_rp_mean_flux_error, phot_rp_mean_flux_over_error, phot_rp_mean_mag, bp_rp, bp_g, g_rp, radial_velocity, radial_velocity_error, l, b,
       ecl_lon, ecl_lat, has_xp_continuous, has_xp_sampled, has_rvs, has_epoch_photometry, has_epoch_rv, non_single_star, pmra, pmra_error, pmdec, pmdec_error
    FROM gaiadr3.gaia_source
    WHERE source_id IN {id_lst}
    '''.format(id_lst = tuple(table_0['source_id'].data))
job1 = Gaia.launch_job(query= query1)
table_1 = job1.get_results()
if 'SOURCE_ID' in table_1.colnames:
    table_1.rename_column('SOURCE_ID','source_id')

table_1 = table.join(table_0,table_1,keys=['source_id'],join_type='left')

## query nss for all soruces in table_0 - get relevant columns

query2 = '''SELECT source_id, parallax, parallax_error , pmra, pmra_error,
 pmdec, pmdec_error, a_thiele_innes, a_thiele_innes_error, b_thiele_innes, b_thiele_innes_error, f_thiele_innes, f_thiele_innes_error,
  g_thiele_innes, g_thiele_innes_error, c_thiele_innes, c_thiele_innes_error, h_thiele_innes, h_thiele_innes_error, period, period_error,
   t_periastron, t_periastron_error, eccentricity, eccentricity_error, center_of_mass_velocity, center_of_mass_velocity_error,
    semi_amplitude_primary, semi_amplitude_primary_error, semi_amplitude_secondary, semi_amplitude_secondary_error, mass_ratio,
     mass_ratio_error, fill_factor_primary, fill_factor_primary_error, fill_factor_secondary, fill_factor_secondary_error, inclination,
      inclination_error, arg_periastron, arg_periastron_error, temperature_ratio, temperature_ratio_error, temperature_ratio_definition, 
      bit_index, corr_vec, goodness_of_fit, efficiency, significance, flags, g_luminosity_ratio, astrometric_jitter, nss_solution_type
    FROM gaiadr3.nss_two_body_orbit
    WHERE source_id IN {id_lst}
    '''.format(id_lst = tuple(table_0['source_id'].data))
job2 = Gaia.launch_job(query= query2)
table_2 = job2.get_results()
if 'SOURCE_ID' in table_2.colnames:
    table_2.rename_column('SOURCE_ID','source_id')

## join tables, reorder columns by importance

table_0 = table.join(table_2,table_1,keys=['source_id'],join_type='left')

# ## take parallax from NSS if available, otherwise take from Gaia source
par_col = [pgaia if np.ma.is_masked(pnss) else pnss for pnss,pgaia in zip(table_0['parallax_1'],table_0['parallax_2'])]
par_err = [egaia if np.ma.is_masked(enss) else enss for enss,egaia in zip(table_0['parallax_error_1'],table_0['parallax_error_2'])]
pmra_col = [pgaia if np.ma.is_masked(pnss) else pnss for pnss,pgaia in zip(table_0['pmra_1'],table_0['pmra_2'])]
pmra_err = [egaia if np.ma.is_masked(enss) else enss for enss,egaia in zip(table_0['pmra_error_1'],table_0['pmra_error_2'])]
pmdec_col = [pgaia if np.ma.is_masked(pnss) else pnss for pnss,pgaia in zip(table_0['pmdec_1'],table_0['pmdec_2'])]
pmdec_err = [egaia if np.ma.is_masked(enss) else enss for enss,egaia in zip(table_0['pmdec_error_1'],table_0['pmdec_error_2'])]
par_col = Table.Column(par_col, name='parallax',unit='mas')
par_err = Table.Column(par_err, name='parallax_error',unit='mas')
pmra_col = Table.Column(pmra_col, name='pmra',unit='mas/yr')
pmra_err = Table.Column(pmra_err, name='pmra_error',unit='mas/yr')
pmdec_col = Table.Column(pmdec_col, name='pmdec',unit='mas/yr')
pmdec_err = Table.Column(pmdec_err, name='pmdec_error',unit='mas/yr')
table_0.add_columns([par_col,par_err,pmra_col,pmra_err,pmdec_col,pmdec_err])
table_0.remove_columns(['parallax_1','parallax_2','parallax_error_1','parallax_error_2','pmra_1','pmra_2','pmra_error_1','pmra_error_2','pmdec_1','pmdec_2','pmdec_error_1','pmdec_error_2'])

table_0['parallax_over_error'] = table_0['parallax']/table_0['parallax_error']
table_0['mg'] = table_0["phot_g_mean_mag"] + 5 * np.log10(table_0["parallax"]) - 10
cols = table_0.colnames
first_cols = ['source_id','cluster','ra','dec','parallax','phot_g_mean_mag','bp_rp']
for c in first_cols:
    cols.remove(c)
cols = first_cols + cols
table_0 = table_0[cols]

# Metallicity
* Convert Zhang metallicity catalog h5 files to fits
* Look for our candidates in the metallicity catalog
* Also calculate the cluster metallicity from the catalog

In [8]:
## convert metallicity catalog h5 files to fits

# import h5py 
# from astropy.table import Table
# import os
# from glob import glob

# # Input location
# # catalog_h5_list = glob('c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/data/metallicity/stellar_params_catalog_*.h5')
# catalog_h5_list = glob('/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/metallicity/stellar_params_catalog_*.h5')
# catalog_h5_list.sort()

# for catalog_h5_loc in catalog_h5_list:
#     print(f"Loading {catalog_h5_loc}")
#     output_fits = Table()
#     with h5py.File(catalog_h5_loc, 'r') as f:
#         for i, key in enumerate(f.keys()):
#             print(f"Loading {i+1}/{len(f.keys())}: {key}")
#             output_fits[key] = f[key][:]
#     base_fn,_ = os.path.splitext(catalog_h5_loc)
#     catalog_fits_loc = base_fn + '.fits'
#     print(f"Saving to {catalog_fits_loc}")
#     output_fits.write(catalog_fits_loc)

In [21]:
## read first metallicity catalog file, create metallicity column, remove other columns

# ColDefs(name = 'chi2_opt'; format = 'E'
#     name = 'dec'; format = 'E'
#     name = 'feh_confidence'; format = 'E'
#     name = 'gdr3_source_id'; format = 'K'
#     name = 'ln_prior'; format = 'E'
#     name = 'logg_confidence'; format = 'E'
#     name = 'quality_flags'; format = 'B'
#     name = 'ra'; format = 'E'
#     name = 'stellar_params_err'; format = '5E'; dim = '(5)'
#     name = 'stellar_params_est'; format = '5E'; dim = '(5)'
#     name = 'teff_confidence'; format = 'E') 

# linux_path = '/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity'
# win_path = 'c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity'

# filepath = os.path.join(linux_path,'stellar_params_catalog_00.fits')
# # filepath = os.path.join(win_path,'stellar_params_catalog_00.fits')

# hdul_7 = fits.open(filepath)
# coldef = fits.ColDefs([hdul_7[1].columns[3],hdul_7[1].columns[8],hdul_7[1].columns[9],hdul_7[1].columns[6]])
# table_7 = Table(hdul_7[1].from_columns(coldef).data)
# table_7.add_column(table_7['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
# table_7.add_column(table_7['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
# table_7.remove_columns(['stellar_params_err','stellar_params_est'])
# table_7.rename_column('gdr3_source_id','source_id')
# hdul_7.close()

# ## read all metallicity catalog files, create metallicity column, remove other columns, append to table_7
# for i in range(1,10):
#     filename = os.path.join(linux_path,f'stellar_params_catalog_0{i}.fits')
#     # filename = os.path.join(win_path,f'stellar_params_catalog_0{i}.fits')
#     print(filename)
#     hdul_temp = fits.open(filename)
#     coldef = fits.ColDefs([hdul_temp[1].columns[3],hdul_temp[1].columns[8],hdul_temp[1].columns[9],hdul_temp[1].columns[6]])
#     table_temp = Table(hdul_temp[1].from_columns(coldef).data)
#     hdul_temp.close()
#     table_temp.add_column(table_temp['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
#     table_temp.add_column(table_temp['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
#     table_temp.remove_columns(['stellar_params_err','stellar_params_est'])
#     table_temp.rename_column('gdr3_source_id','source_id')
#     table_7 = table.vstack([table_7,table_temp])

# table_0 = table.join(table_0,table_7,keys=['source_id'],join_type='left')

/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_01.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_02.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_03.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_04.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_05.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_06.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_07.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_08.fits
/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_09.fits


In [ ]:
# ## get metallicity for all cluster members, take median

# linux_path = '/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity'
# win_path = 'c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity'

# filepath = os.path.join(linux_path,'stellar_params_catalog_00.fits')
# # filepath = os.path.join(win_path,'stellar_params_catalog_00.fits')

# hdul_7 = fits.open(filepath)
# coldef = fits.ColDefs([hdul_7[1].columns[3],hdul_7[1].columns[8],hdul_7[1].columns[9],hdul_7[1].columns[6]])
# table_7 = Table(hdul_7[1].from_columns(coldef).data)
# table_7.add_column(table_7['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
# table_7.add_column(table_7['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
# table_7.remove_columns(['stellar_params_err','stellar_params_est'])
# table_7.rename_column('gdr3_source_id','source_id')
# hdul_7.close()

# ## read all metallicity catalog files, create metallicity column, remove other columns, append to table_7
# for i in range(1,10):
#     filename = os.path.join(linux_path,f'stellar_params_catalog_0{i}.fits')
#     # filename = os.path.join(win_path,f'stellar_params_catalog_0{i}.fits')
#     print(filename)
#     hdul_temp = fits.open(filename)
#     coldef = fits.ColDefs([hdul_temp[1].columns[3],hdul_temp[1].columns[8],hdul_temp[1].columns[9],hdul_temp[1].columns[6]])
#     table_temp = Table(hdul_temp[1].from_columns(coldef).data)
#     hdul_temp.close()
#     table_temp.add_column(table_temp['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
#     table_temp.add_column(table_temp['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
#     table_temp.remove_columns(['stellar_params_err','stellar_params_est'])
#     table_temp.rename_column('gdr3_source_id','source_id')
#     table_7 = table.vstack([table_7,table_temp])

# table_7 = table_7[table_7['quality_flags'] < 8]
# table_0['Fe_H_cluster'] = np.full(len(table_0),np.nan)
# table_0['Fe_H_cluster_std'] = np.full(len(table_0),np.nan)

# for i,id in tqdm(enumerate(table_0['id']),total=len(table_0['id'])):
#     nbr = members[members['id']==id]
#     nbr = nbr[nbr['probability'] > 0.99]
#     fh_arr = table_7[np.isin(table_7['source_id'],nbr['source_id'])]['Fe_H_est']
#     table_0[i]['Fe_H_cluster'] = np.median(fh_arr)
#     table_0[i]['Fe_H_cluster_std'] = np.std(fh_arr)

# Extinction
* Query dust maps for reddening estimates
* Northern targets- Bayestar 3D map
* Southern targets- Stilism 3D map

In [ ]:
## query extinction (northern)

# from dustmaps.config import config
# config['data_dir'] = '/path/to/store/maps/in'

# import dustmaps.bayestar
# dustmaps.bayestar.fetch()

# from dustmaps.bayestar import BayestarQuery

# bayestar = BayestarQuery(max_samples=2, version='bayestar2015')
# l = table_0['l']
# b = table_0['b']
# d = np.abs(1000/table_0['parallax'])
# coord = SkyCoord(l=l*u.deg, b=b*u.deg, distance=d*u.pc, frame='galactic')
# ebv = 0.884 * bayestar(coord, mode='median')

In [ ]:
## query extinction (southern)
# from urllib import request
# av = []
# with tqdm(total=len(table_0)) as pbar:
#     for ra,dec,par in zip(table_0['ra'],table_0['dec'],table_0['parallax']):
#         d = np.abs(1000/par)
#         with  request.urlopen(f'https://astro.acri-st.fr/gaia_dev/extinction?frame=icrs&vlong={ra}&ulong=deg&vlat={dec}&ulat=deg&distance={d}') as response:
#             html = response.read()
#             av.append(float(html.split(b'\n')[1].split(b',')[1]))
#         pbar.update(1)

# Galex
* Cross match with galex to get NUV,FUV mag where available

In [34]:
## Astroquery GALEX data
# i = 40
# ra = table_0[i]['ra']
# dec = table_0[i]['dec']
# r_arcsec = 10
# r_deg = r_arcsec/3600
# catalog_data = Catalogs.query_object(f'{ra} {dec}',catalog='Galex',radius = r_deg)

# print(np.min(catalog_data['distance_arcmin'].data)*60)

for c in ['distance_arcmin','nuv_mag','nuv_magerr','fuv_mag','fuv_magerr']:
    table_0[c] = np.full(len(table_0),np.nan)


for i in tqdm(range(len(table_0))):
    ra = table_0[i]['ra']
    dec = table_0[i]['dec']
    r_arcsec = 2
    r_deg = r_arcsec/3600
    catalog_data = Catalogs.query_object(f'{ra} {dec}',catalog='Galex',radius = r_deg)

    if len(catalog_data)>0:
        for c in ['distance_arcmin','nuv_mag','nuv_magerr','fuv_mag','fuv_magerr']:
            table_0[c][i] = catalog_data[c][0]

  1%|          | 4/413 [00:04<07:08,  1.05s/it]/home/oreni/Documents/WD_project/.venv/lib/python3.10/site-packages/astropy/table/column.py:1362: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value
100%|██████████| 413/413 [06:50<00:00,  1.01it/s]


# Aesthetics
* Deal with problematic data types in the table (so we can save it as .fits file)
* Rearrange columns
* Write to .fits

In [35]:
## deal with problematic data types

col = table.Column([str(cvec) for cvec in table_0['corr_vec']],name = 'corr_vec',dtype = str)
table_0['nss_solution_type'] = table_0['nss_solution_type'].astype(str)
table_0['corr_vec'] = col

In [36]:
## sort columns by importance

cols = table_0.colnames
first_cols = ['source_id','ra','dec','parallax','mg','phot_g_mean_mag','bp_rp','age','age_err','av_for_mass_interp','av_err','mh_for_mass_interp','mh_err','nuv_mag']
for c in first_cols:
    cols.remove(c)
cols = first_cols + cols
table_0 = table_0[cols]

In [37]:
## save table
table_0.write('table.fits', format='fits',overwrite=True)